In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk

In [2]:
df = pd.read_csv('Data/vehicles.csv')
print(df.head())
print(df.shape)

           id                                                url  \
0  7184791621  https://duluth.craigslist.org/ctd/d/duluth-200...   
1  7184773187  https://duluth.craigslist.org/cto/d/saginaw-20...   
2  7193375964  https://newhaven.craigslist.org/cto/d/stratfor...   
3  7195108810  https://albuquerque.craigslist.org/cto/d/albuq...   
4  7184712241  https://duluth.craigslist.org/ctd/d/rush-city-...   

              region                          region_url  price    year  \
0  duluth / superior       https://duluth.craigslist.org   6995  2000.0   
1  duluth / superior       https://duluth.craigslist.org   8750  2013.0   
2          new haven     https://newhaven.craigslist.org  10900  2013.0   
3        albuquerque  https://albuquerque.craigslist.org  12500  2003.0   
4  duluth / superior       https://duluth.craigslist.org  16995  2007.0   

  manufacturer                  model  condition    cylinders  ... drive  \
0          gmc        new sierra 1500  excellent  8 cylinders  .

In [3]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'vin', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long'],
      dtype='object')

In [4]:
df = df.drop(columns=['id', 'url', 'region', 'region_url', 'county', 'image_url', 'county','state','lat','long','vin','size','type','description'])

In [5]:
df['model'].value_counts()

f-150                      6073
silverado 1500             3878
escape                     3208
camry                      2906
1500                       2898
                           ... 
1930 aa                       1
250 cargo                     1
pathfinder 4 wd               1
Plymouth coupe                1
accord 4dr v6 auto ex-l       1
Name: model, Length: 27042, dtype: int64

In [6]:
counts = df['model'].value_counts()
to_remove = counts[counts <= 10].index
df = df[~df.model.isin(to_remove)]

In [7]:
df['model'].value_counts()

f-150                         6073
silverado 1500                3878
escape                        3208
camry                         2906
1500                          2898
                              ... 
olet Silverado 1500 LD          11
wrangler unlimited sport s      11
128i convertible                11
4 series 428i xdrive coupe      11
f-350 lariat                    11
Name: model, Length: 3071, dtype: int64

In [8]:
df.shape

(372656, 12)

In [9]:
df['cylinders'].value_counts()

4 cylinders     60030
6 cylinders     58322
8 cylinders     43692
5 cylinders      1238
10 cylinders      655
other             534
3 cylinders       286
12 cylinders       54
Name: cylinders, dtype: int64

In [10]:
df.drop(df[df['cylinders'] == 'other'].index, inplace = True) 

In [11]:
df['cylinders'].value_counts()

4 cylinders     60030
6 cylinders     58322
8 cylinders     43692
5 cylinders      1238
10 cylinders      655
3 cylinders       286
12 cylinders       54
Name: cylinders, dtype: int64

In [12]:
df['title_status'].value_counts()

clean         266601
rebuilt         5136
salvage         2697
lien            1566
missing          427
parts only       156
Name: title_status, dtype: int64

In [13]:
df['transmission'].value_counts()

automatic    249480
manual        17605
other          9437
Name: transmission, dtype: int64

In [14]:
df.drop(df[df['transmission'] == 'other'].index, inplace = True) 

In [15]:
df['transmission'].value_counts()

automatic    249480
manual        17605
Name: transmission, dtype: int64

In [16]:
df['drive'].value_counts()

4wd    81181
fwd    76068
rwd    31291
Name: drive, dtype: int64

In [17]:
df['paint_color'].value_counts()

white     45920
black     35532
silver    27042
blue      18809
red       17877
grey      17653
custom     5390
green      4750
brown      3806
yellow     1286
orange     1112
purple      548
Name: paint_color, dtype: int64

In [18]:
df.drop(df[df['paint_color'] == 'custom'].index, inplace = True) 

In [19]:
df['paint_color'].value_counts()

white     45920
black     35532
silver    27042
blue      18809
red       17877
grey      17653
green      4750
brown      3806
yellow     1286
orange     1112
purple      548
Name: paint_color, dtype: int64

In [20]:
df['paint_color'] == 'Nan'

1         False
2         False
3         False
5         False
6         False
          ...  
423852    False
423853    False
423854    False
423855    False
423856    False
Name: paint_color, Length: 357295, dtype: bool

In [21]:
df.shape

(357295, 12)

In [22]:
df['fuel'].value_counts()

gas         232736
diesel       17898
other         7909
hybrid        2939
electric       481
Name: fuel, dtype: int64

In [23]:
df.drop(df[df['fuel'] == 'other'].index, inplace = True) 

In [24]:
df['fuel'].value_counts()

gas         232736
diesel       17898
hybrid        2939
electric       481
Name: fuel, dtype: int64

In [25]:
df['condition'].value_counts()


excellent    69093
good         44685
like new     13725
fair          5170
new            723
salvage        371
Name: condition, dtype: int64

In [26]:
df['manufacturer'].value_counts()

ford               47784
chevrolet          37955
toyota             20960
honda              16051
nissan             14356
jeep               11685
gmc                 9850
dodge               9823
ram                 9436
hyundai             7320
subaru              6414
kia                 5804
bmw                 5594
volkswagen          5472
chrysler            4980
mercedes-benz       4678
buick               3643
cadillac            3512
lexus               2952
mazda               2874
audi                2232
acura               2198
lincoln             2057
pontiac             1917
infiniti            1703
mitsubishi          1412
volvo               1339
mercury             1093
mini                 981
rover                898
saturn               861
jaguar               510
fiat                 380
tesla                134
alfa-romeo            65
datsun                47
harley-davidson       47
ferrari               14
aston-martin          11
land rover             8


In [27]:
df['year'].value_counts()

2017.0    23654
2013.0    17077
2015.0    17054
2014.0    17022
2016.0    16581
          ...  
1945.0        1
1922.0        1
1921.0        1
1920.0        1
1913.0        1
Name: year, Length: 102, dtype: int64

In [28]:
df['price'].value_counts()

0        29182
3500      3319
8995      3167
7995      3056
6995      3012
         ...  
14538        1
3730         1
9447         1
42631        1
9913         1
Name: price, Length: 14609, dtype: int64

In [29]:
df.drop(df[df['price'] == 0].index, inplace = True) 

In [30]:
df['price'].value_counts()

3500     3319
8995     3167
7995     3056
6995     3012
5995     2990
         ... 
26945       1
11133       1
17840       1
888         1
12871       1
Name: price, Length: 14608, dtype: int64

In [31]:
df.shape

(320204, 12)

In [32]:
df = df.dropna()
df.shape

(69923, 12)

In [33]:
file = open('Data/vehicles_new', 'wb')
pk.dump(df, file)